# Improved Simulation of Stabilizer Codes

In [1]:
from qiskit import *
import numpy as np
from qiskit.providers.aer.extensions import *

# The problem

$\left(\begin{matrix}
\hat{Z} & \mathbb{I} & \mathbb{I} & \mathbb{I} & \mathbb{I} & \mathbb{I} & \mathbb{I} \\
\mathbb{I} & \hat{Z} & \mathbb{I} & \mathbb{I} & \mathbb{I} & \mathbb{I} & \mathbb{I} \\
\mathbb{I} & \mathbb{I} & \hat{Z} & \mathbb{I} & \mathbb{I} & \mathbb{I} & \mathbb{I} \\
\mathbb{I} & \mathbb{I} & \mathbb{I} & \hat{Z} & \mathbb{I} & \mathbb{I} & \mathbb{I} \\
\mathbb{I} & \mathbb{I} & \mathbb{I} & \mathbb{I} & \hat{Z} & \mathbb{I} & \mathbb{I} \\
\mathbb{I} & \mathbb{I} & \mathbb{I} & \mathbb{I} & \mathbb{I} & \hat{Z} & \mathbb{I} \\
\mathbb{I} & \mathbb{I} & \mathbb{I} & \mathbb{I} & \mathbb{I} & \mathbb{I} & \hat{Z} 
\end{matrix}\right) \Longrightarrow
\left(\begin{matrix}
\hat{X} & \hat{X} & \hat{X} & \hat{X} & \mathbb{I} & \mathbb{I} & \mathbb{I} \\
\mathbb{I} & \hat{X} & \hat{X} & \mathbb{I} &  \hat{X} & \hat{X} & \mathbb{I} \\
\mathbb{I}& \mathbb{I} & \hat{X} & \hat{X} & \mathbb{I} & \hat{X} & \hat{X} \\
\hat{Z} & \hat{Z} & \hat{Z} & \hat{Z} & \mathbb{I} & \mathbb{I} & \mathbb{I} \\
\mathbb{I} & \hat{Z} & \hat{Z} & \mathbb{I} &  \hat{Z} & \hat{Z} & \mathbb{I} \\
\mathbb{I}& \mathbb{I} & \hat{Z} & \hat{Z} & \mathbb{I} & \hat{Z} & \hat{Z} \\
\hat{X} &\hat{X} &\hat{X} &\hat{X} &\hat{X} &\hat{X} &\hat{X}
\end{matrix} \right)
$

# Our Solution

## Adding the option of specifying an initial stabilizer state.

In [73]:
class Initialize_Stabilizer(Instruction):
    def __init__(self, stabs):
        num_qubits = int (stabs.shape[1]/2)
        
        #stabs.append(destabs) 
        stab_string=[]
        for i in range(stabs.shape[0]):
            temp=''
            #convert to string
            for j in range(0,num_qubits): 
                qi=[stabs[i,j],stabs[i,j+num_qubits]]
                if qi==[0,0]:
                    temp=temp+'I'
                elif qi==[0,1]:
                    temp= temp+'Z'
                elif qi==[1,0]:
                    temp=temp+'X'
                else:
                    temp=temp+'Y'
    
            stab_string.append(temp)#+'IIIIII')
        
        
        
        param = np.array(stab_string)    # np.array is a hack to make non-float params work
        super().__init__("initialize_stabilizer", num_qubits, 0, [param])

# Steane Code Example

![title](Capture.PNG)

In [72]:
stabilizers=np.array([[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                     [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0],
                     [0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
                     [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0], 
                     [0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0],
                     [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1],
                     [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]])
destabilizers=np.array([[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
                     [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
                     [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
                     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
                     [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
                     [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]])
stabs_combined=np.concatenate((stabilizers,destabilizers), axis=0)

# No Error

In [167]:
qubits=QuantumRegister(7)
syndrome=ClassicalRegister(7)

circ = QuantumCircuit(qubits, syndrome)


instr=Initialize_Stabilizer(stabs_combined)
circ.snapshot_stabilizer('pre_init')
circ.append(instr, qubits)
circ.snapshot_stabilizer('post_init')


#apply an X error on qubit 1, expect Z stabiliser 1 to give 1
#circ.x(qubits[2])

#measure
for i in range(7):
    circ.measure(qubits[i],syndrome[i])
    


#run job
opts = {"method": "stabilizer"}
sim = Aer.get_backend("qasm_simulator")
job = execute(circ, sim, shots=1,basis_gates=['initialize_stabilizer','cx','h','x','z'], backend_options=opts)
results = job.result()
#results.get_counts(0)

counts=results.get_counts(0)
state=[]
for i in range(len(counts)):
    state.append(list(counts.popitem()))
    
stab1=[]
stab2=[]
stab3=[]
for j in range(len(state)):
    state_list=[int(i) for i in state[j][0]]
    #state_list=state_list[::-1]
    stab1.append(sum([state_list[i] for i in [0,1,2,3]]) % 2)
    stab2.append(sum([state_list[i] for i in [1,2,4,5]]) % 2)
    stab3.append(sum([state_list[i] for i in [2,3,5,6]]) % 2)

print("Red Z plaquette:",stab1[0])
print("Blue Z plaquette:",stab2[0])
print("Green Z plaquette:",stab3[0])

Red Z plaquette: 0
Blue Z plaquette: 0
Green Z plaquette: 0


# X Error on Qubit 1

In [168]:
qubits=QuantumRegister(7)
syndrome=ClassicalRegister(7)

circ = QuantumCircuit(qubits, syndrome)


instr=Initialize_Stabilizer(stabs_combined)
circ.snapshot_stabilizer('pre_init')
circ.append(instr, qubits)
circ.snapshot_stabilizer('post_init')


#apply an X error on qubit 1, expect Z stabiliser 1 to give 1
circ.x(qubits[6])

#measure
for i in range(7):
    circ.measure(qubits[i],syndrome[i])
    


#run job
opts = {"method": "stabilizer"}
sim = Aer.get_backend("qasm_simulator")
job = execute(circ, sim, shots=1,basis_gates=['initialize_stabilizer','cx','h','x','z'], backend_options=opts)
results = job.result()
#results.get_counts(0)

counts=results.get_counts(0)
state=[]
for i in range(len(counts)):
    state.append(list(counts.popitem()))
    
stab1=[]
stab2=[]
stab3=[]
for j in range(len(state)):
    state_list=[int(i) for i in state[j][0]]
    #state_list=state_list[::-1]
    stab1.append(sum([state_list[i] for i in [0,1,2,3]]) % 2)
    stab2.append(sum([state_list[i] for i in [1,2,4,5]]) % 2)
    stab3.append(sum([state_list[i] for i in [2,3,5,6]]) % 2)

print("Red Z plaquette:",stab1[0])
print("Blue Z plaquette:",stab2[0])
print("Green Z plaquette:",stab3[0])

Red Z plaquette: 1
Blue Z plaquette: 0
Green Z plaquette: 0


In [77]:
results.data(0)['snapshots']['stabilizer']

{'post_init': [['XXXXIII',
   'ZZZZIII',
   'IXXIXXI',
   'IZZIZZI',
   'IIXXIXX',
   'IIZZIZZ',
   'XXXXXXX']],
 'pre_init': [['IIIIIIZ',
   'IIIIIZI',
   'IIIIZII',
   'IIIZIII',
   'IIZIIII',
   'IZIIIII',
   'ZIIIIII']]}

In [5]:
job.result().to_dict()

{'status': 'COMPLETED',
 'job_id': '352a9394-7ea8-4395-980a-06d11709fda3',
 'results': [{'status': 'DONE',
   'shots': 1024,
   'data': {},
   'success': True,
   'meas_level': 2,
   'header': {'qreg_sizes': [['q', 2]],
    'clbit_labels': [],
    'memory_slots': 0,
    'qubit_labels': [['q', 0], ['q', 1]],
    'name': 'circuit0',
    'n_qubits': 2,
    'creg_sizes': []},
   'metadata': {'measure_sampling': True,
    'method': 'stabilizer',
    'parallel_shots': 1,
    'parallel_state_update': 1},
   'seed_simulator': 90643069,
   'time_taken': 0.000410275}],
 'header': {'backend_name': 'qasm_simulator', 'backend_version': '0.4.0'},
 'success': True,
 'backend_name': 'qasm_simulator',
 'date': '2019-11-20T10:17:11.293121',
 'qobj_id': 'e8489968-0093-4b5d-9165-a6d680d57ebc',
 'backend_version': '0.4.0',
 'metadata': {'max_memory_mb': 5927,
  'omp_enabled': True,
  'parallel_experiments': 1,
  'time_taken': 0.002061195},
 'time_taken': 0.027180194854736328}

In [6]:
assemble(qc).experiments[0].instructions

[QasmQobjInstruction(name='initialize_stabilizer', params=[array(['XI', 'IX', 'ZI', 'IZ'], dtype='<U2')], qubits=[0, 1])]

In [ ]:
#std::vector<std::vector<std::string>> tmp = js["params"]
#op.string_params = tmp[0]